# 🍯 Classificador de Riscos - Produção de Mel

### Sistema de classificação inspirado no dataset Iris, adaptado para segurança alimentar

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Configurar visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Geração do Dataset Sintético

In [ ]:
def gerar_dataset_mel(n_samples=1000):
    """
    Gera dataset sintético com 18 features para classificação de riscos
    """
    np.random.seed(42)
    data = []
    
    for i in range(n_samples):
        # Features numéricas
        temp_proc = np.random.uniform(15, 85)
        umidade = np.random.uniform(30, 90)
        pH = np.random.uniform(3.2, 6.5)
        tempo_exp = np.random.uniform(0, 240)
        temp_armaz = np.random.uniform(10, 40)
        contagem_micro = np.random.uniform(0, 10000)
        atividade_agua = np.random.uniform(0.4, 0.8)
        hmf = np.random.uniform(0, 100)
        
        # Features categóricas
        tipo_embalagem = np.random.choice([1, 2, 3])
        higiene = np.random.choice([1, 2, 3, 4])
        cond_equip = np.random.choice([1, 2, 3, 4])
        controle_pragas = np.random.choice([1, 2, 3, 4])
        fonte_agua = np.random.choice([1, 2, 3])
        
        # Features binárias
        filtrado = np.random.choice([0, 1])
        pasteurizado = np.random.choice([0, 1])
        cert_organica = np.random.choice([0, 1])
        uso_antibio = np.random.choice([0, 1])
        rastreabilidade = np.random.choice([0, 1])
        
        # Calcular risco
        risco_score = 0
        
        if temp_proc > 70 or temp_proc < 20: risco_score += 3
        if pH < 3.5 or pH > 6.0: risco_score += 3
        if contagem_micro > 5000: risco_score += 4
        if higiene <= 2: risco_score += 3
        if hmf > 60: risco_score += 3
        if umidade > 70: risco_score += 1
        if tempo_exp > 120: risco_score += 1
        if atividade_agua > 0.65: risco_score += 2
        if controle_pragas <= 2: risco_score += 1
        if cert_organica == 1: risco_score -= 1
        if rastreabilidade == 1: risco_score -= 1
        if pasteurizado == 1: risco_score -= 2
        if higiene >= 3: risco_score -= 1
        
        if risco_score >= 8:
            risco = 2  # ALTO
        elif risco_score >= 4:
            risco = 1  # MÉDIO
        else:
            risco = 0  # BAIXO
        
        if np.random.random() < 0.1:
            risco = np.random.choice([0, 1, 2])
        
        data.append({
            'temperatura_processo': temp_proc,
            'umidade_relativa': umidade,
            'pH_mel': pH,
            'tempo_exposicao': tempo_exp,
            'temperatura_armazenamento': temp_armaz,
            'contagem_microbiana': contagem_micro,
            'atividade_agua': atividade_agua,
            'hidroximetilfurfural': hmf,
            'tipo_embalagem': tipo_embalagem,
            'higiene_manipulador': higiene,
            'condicao_equipamento': cond_equip,
            'controle_pragas': controle_pragas,
            'fonte_agua': fonte_agua,
            'filtrado': filtrado,
            'pasteurizado': pasteurizado,
            'certificacao_organica': cert_organica,
            'uso_antibioticos': uso_antibio,
            'rastreabilidade': rastreabilidade,
            'risco': risco
        })
    
    return pd.DataFrame(data)

# Gerar dataset
df = gerar_dataset_mel(1500)
print(f"Dataset gerado: {df.shape[0]} amostras, {df.shape[1]-1} features")
df.head()

## 2. Análise Exploratória

In [ ]:
# Distribuição das classes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico de barras
risco_counts = df['risco'].value_counts().sort_index()
labels = ['BAIXO', 'MÉDIO', 'ALTO']
colors = ['green', 'yellow', 'red']

axes[0].bar(labels, risco_counts.values, color=colors, alpha=0.7)
axes[0].set_title('Distribuição das Classes de Risco')
axes[0].set_xlabel('Classe de Risco')
axes[0].set_ylabel('Quantidade')

# Gráfico de pizza
axes[1].pie(risco_counts.values, labels=labels, colors=colors, 
            autopct='%1.1f%%', startangle=90)
axes[1].set_title('Proporção das Classes')

plt.tight_layout()
plt.show()

print("\nDistribuição detalhada:")
for i, label in enumerate(labels):
    count = risco_counts.iloc[i]
    pct = (count / len(df)) * 100
    print(f"{label}: {count} amostras ({pct:.1f}%)")

In [ ]:
# Análise das features numéricas por classe de risco
numeric_features = ['temperatura_processo', 'pH_mel', 'contagem_microbiana', 
                   'atividade_agua', 'hidroximetilfurfural']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(numeric_features):
    for risco in [0, 1, 2]:
        data = df[df['risco'] == risco][feature]
        axes[idx].hist(data, alpha=0.5, label=labels[risco], 
                      color=colors[risco], bins=20)
    
    axes[idx].set_title(f'Distribuição: {feature}')
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frequência')
    axes[idx].legend()

# Remover eixo extra
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## 3. Preparação dos Dados

In [ ]:
# Separar features e target
X = df.drop('risco', axis=1)
y = df['risco']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Normalizar dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Conjunto de treino: {X_train.shape[0]} amostras")
print(f"Conjunto de teste: {X_test.shape[0]} amostras")

## 4. Treinamento do Modelo

In [ ]:
# Treinar Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)

# Predições
y_pred = rf_model.predict(X_test_scaled)
y_pred_proba = rf_model.predict_proba(X_test_scaled)

print("Modelo treinado com sucesso!")

## 5. Avaliação do Modelo

In [ ]:
# Relatório de classificação
print("="*60)
print("RELATÓRIO DE CLASSIFICAÇÃO")
print("="*60)
print(classification_report(y_test, y_pred, target_names=labels))

# Cross-validation
cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5)
print(f"\nCross-validation Score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Matriz de Confusão')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

In [ ]:
# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:10], 
         feature_importance['importance'][:10])
plt.xlabel('Importância')
plt.title('Top 10 Features Mais Importantes')
plt.gca().invert_yaxis()
plt.show()

print("\nTop 5 features:")
print(feature_importance.head())

## 6. Visualização com PCA

In [ ]:
# Redução de dimensionalidade com PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=y_train, cmap='viridis', 
                     alpha=0.6, edgecolors='black', linewidth=0.5)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variância)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variância)')
plt.title('Visualização do Dataset com PCA')
plt.colorbar(scatter, label='Classe de Risco', ticks=[0, 1, 2])
plt.grid(True, alpha=0.3)
plt.show()

print(f"Variância explicada: {sum(pca.explained_variance_ratio_):.2%}")

## 7. Comparação com Outros Modelos

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Definir modelos
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

# Treinar e avaliar cada modelo
results = []

for name, model in models.items():
    # Treinar
    model.fit(X_train_scaled, y_train)
    
    # Prever
    y_pred = model.predict(X_test_scaled)
    
    # Calcular métricas
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    results.append({
        'Modelo': name,
        'Acurácia': accuracy,
        'Precisão': precision,
        'Recall': recall,
        'F1-Score': f1
    })

# Criar DataFrame com resultados
results_df = pd.DataFrame(results).sort_values('Acurácia', ascending=False)

# Visualizar comparação
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(results_df))
width = 0.2

metrics = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
colors_metrics = ['blue', 'green', 'orange', 'red']

for i, metric in enumerate(metrics):
    ax.bar(x + i*width, results_df[metric], width, 
           label=metric, color=colors_metrics[i], alpha=0.7)

ax.set_xlabel('Modelo')
ax.set_ylabel('Score')
ax.set_title('Comparação de Modelos de Classificação')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(results_df['Modelo'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTabela de Resultados:")
print(results_df.to_string(index=False))

## 8. Simulação de Predição em Tempo Real

In [ ]:
def prever_risco_interativo(modelo, scaler):
    """
    Interface interativa para predição de risco
    """
    print("="*60)
    print("SISTEMA DE PREDIÇÃO DE RISCO - PRODUÇÃO DE MEL")
    print("="*60)
    print("\nInsira os valores das features:\n")
    
    # Simular entrada de dados (em produção seria input do usuário)
    amostra = {
        'temperatura_processo': 45,
        'umidade_relativa': 65,
        'pH_mel': 4.2,
        'tempo_exposicao': 90,
        'temperatura_armazenamento': 25,
        'contagem_microbiana': 2500,
        'atividade_agua': 0.55,
        'hidroximetilfurfural': 35,
        'tipo_embalagem': 2,
        'higiene_manipulador': 3,
        'condicao_equipamento': 3,
        'controle_pragas': 3,
        'fonte_agua': 2,
        'filtrado': 1,
        'pasteurizado': 0,
        'certificacao_organica': 1,
        'uso_antibioticos': 0,
        'rastreabilidade': 1
    }
    
    # Criar DataFrame
    amostra_df = pd.DataFrame([amostra])
    
    # Normalizar
    amostra_scaled = scaler.transform(amostra_df)
    
    # Prever
    predicao = modelo.predict(amostra_scaled)[0]
    probabilidades = modelo.predict_proba(amostra_scaled)[0]
    
    # Mostrar resultados
    print("\n" + "="*60)
    print("RESULTADO DA ANÁLISE")
    print("="*60)
    
    risco_label = labels[predicao]
    risco_cor = colors[predicao]
    
    print(f"\n🎯 Classificação: {risco_label}")
    print("\nProbabilidades:")
    for i, label in enumerate(labels):
        print(f"  {label}: {probabilidades[i]:.2%}")
    
    # Visualizar probabilidades
    plt.figure(figsize=(8, 4))
    plt.bar(labels, probabilidades, color=colors, alpha=0.7)
    plt.title('Probabilidade de Cada Classe de Risco')
    plt.ylabel('Probabilidade')
    plt.ylim(0, 1)
    
    for i, v in enumerate(probabilidades):
        plt.text(i, v + 0.01, f'{v:.2%}', ha='center')
    
    plt.show()
    
    # Recomendações baseadas no risco
    print("\n📋 RECOMENDAÇÕES:")
    if predicao == 2:  # ALTO
        print("⚠️ ATENÇÃO URGENTE NECESSÁRIA!")
        print("- Revisar processo de produção imediatamente")
        print("- Verificar conformidade com padrões de segurança")
        print("- Implementar medidas corretivas")
    elif predicao == 1:  # MÉDIO
        print("⚡ Monitoramento recomendado")
        print("- Identificar pontos de melhoria")
        print("- Intensificar controles preventivos")
    else:  # BAIXO
        print("✅ Processo dentro dos padrões")
        print("- Manter boas práticas atuais")
        print("- Continuar monitoramento regular")

# Executar predição
prever_risco_interativo(rf_model, scaler)

## 9. Salvando o Modelo

In [ ]:
import joblib

# Salvar modelo e scaler
joblib.dump(rf_model, 'modelo_risco_mel.pkl')
joblib.dump(scaler, 'scaler_risco_mel.pkl')

print("✅ Modelo salvo como 'modelo_risco_mel.pkl'")
print("✅ Scaler salvo como 'scaler_risco_mel.pkl'")

# Exemplo de como carregar
# modelo_carregado = joblib.load('modelo_risco_mel.pkl')
# scaler_carregado = joblib.load('scaler_risco_mel.pkl')